In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env 
load_dotenv()

api_key = os.getenv('ALPACA_API_KEY')
secret_key = os.getenv('ALPACA_SECRET_KEY')
paper = True

In [6]:
from datetime import datetime, timedelta

import alpaca
from alpaca.data.historical.option import OptionHistoricalDataClient, OptionLatestQuoteRequest
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.trading.client import TradingClient, GetAssetsRequest
from alpaca.trading.requests import GetOptionContractsRequest, LimitOrderRequest, MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import AssetStatus, ContractType, OrderSide, OrderType, TimeInForce, QueryOrderStatus


In [7]:
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper)

stock_data_client = StockHistoricalDataClient(api_key=api_key, secret_key=secret_key)

option_data_client = OptionHistoricalDataClient(api_key=api_key, secret_key=secret_key)

In [8]:
acct = trade_client.get_account()

print(f"Options Approved Level: {acct.options_approved_level}")
print(f"Options Trading Level: {acct.options_trading_level}")
print(f"Options Buying Power: {acct.options_buying_power}")

Options Approved Level: 2
Options Trading Level: 2
Options Buying Power: 100000


In [10]:
# Risk management
acct_config = trade_client.get_account_configurations()
print(f"Max Options Trading Level: {acct_config.max_options_trading_level}")

acct_config.max_options_trading_level = 1
trade_client.set_account_configurations(acct_config)

acct_config = trade_client.get_account_configurations()
print(f"Max Options Trading Level: {acct_config.max_options_trading_level}")

Max Options Trading Level: None
Max Options Trading Level: 1


In [13]:
# List of assets to have options available
req = GetAssetsRequest(
    status = AssetStatus.ACTIVE, 
    attributes = "options_enabled"
)

options_enabled_underlyings = trade_client.get_all_assets(req)
print(f"Number of Underlyings with Options: {len(options_enabled_underlyings)}")

underlying_symbol = "TSLA"
underlying = trade_client.get_asset(symbol_or_asset_id=underlying_symbol)
print(f"{underlying_symbol} has options? {'options_enabled' in underlying.attributes}")



Number of Underlyings with Options: 5773
TSLA has options? False


In [14]:
# Call Overwrite
# 
# What is currently in the account?
positions = trade_client.get_all_positions()

for p in positions:
    print(p)





In [ ]:
# Overwrite on TSLA

# Query all call options based on these filters
today = datetime.now().date()
min_expiration = today + timedelta(days=14)
max_expiration = today + timedelta(days=90)

underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=underlying_symbol)
underlying_latest_trade = stock_data_client.get_stock_latest_trade(underlying_trade_request)
underlying_last_price = underlying_latest_trade[underlying_symbol].price

print(f"{underlying} last trade price: {underlying_last_price}")

min_strike = str(round(underlying_last_price * 1.02, 2))

print(f"Min Expiration: {min_expiration}")
print(f"Max Expiration: {max_expiration}")

